# "Unit test" :) 

In [8]:

DB_ULR = "neo4j://localhost:7687"
DB_USER = "neo4j"
DB_PASS = "test1234"
DB_NAME = "vector"
index_name = 'myVectorIndex'

In [9]:
import pandas as pd
from neo4j import GraphDatabase, RoutingControl # Python database driver

In [10]:
driver = GraphDatabase.driver(DB_ULR, auth=(DB_USER, DB_PASS))
driver.verify_connectivity()

In [11]:
driver.execute_query(
    'create database {dbname} if not exists'.format(dbname = DB_NAME),
    None,
    RoutingControl.WRITE,
    database_= 'system'
)

EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0x146302be0>, keys=[])

In [12]:
driver.execute_query(
    'call db.index.vector.createNodeIndex($index_name, "WithVector", "vector", 2, "cosine")',
    {'index_name': index_name},
    routing_= RoutingControl.WRITE,
    database_= DB_NAME
)

Unable to retrieve routing information
Transaction failed and will be retried in 0.9575359276078246s (Unable to retrieve routing information)


EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0x28b0e6220>, keys=[])

In [13]:
records, summary, keys = driver.execute_query(
    'show indexes yield * where name = $index_name return *',
    {'index_name': index_name},
    routing_= RoutingControl.READ,
    database_= DB_NAME
)
for record in records:
    print(record)

<Record id=3 name='myVectorIndex' state='ONLINE' populationPercent=100.0 type='VECTOR' entityType='NODE' labelsOrTypes=['WithVector'] properties=['vector'] indexProvider='vector-1.0' owningConstraint=None lastRead=None readCount=None trackedSince=None options={'indexProvider': 'vector-1.0', 'indexConfig': {'vector.dimensions': 2, 'vector.similarity_function': 'cosine'}} failureMessage='' createStatement="CALL db.index.vector.createNodeIndex('myVectorIndex', 'WithVector', 'vector', 2, 'COSINE')">


In [14]:
records, summary, keys = driver.execute_query(
    '''
    unwind $data as row 
    create (n:WithVector{id: row.id}) with n, row
    call db.create.setNodeVectorProperty(n, 'vector', row.vector)
    return n
    ''',
    data=[
            {'id': 1, 'vector':[1.0, 0.0]},
            {'id': 2, 'vector':[1.0, 1.0]},
            {'id': 3, 'vector':[0.0, 1.0]},
        ],
    routing_= RoutingControl.WRITE,
    database_= DB_NAME
)
for record in records:
    print(record)

<Record n=<Node element_id='4:f0adc280-ec80-4264-96b8-147afa2d1ff1:0' labels=frozenset({'WithVector'}) properties={'vector': [1.0, 0.0], 'id': 1}>>
<Record n=<Node element_id='4:f0adc280-ec80-4264-96b8-147afa2d1ff1:1' labels=frozenset({'WithVector'}) properties={'vector': [1.0, 1.0], 'id': 2}>>
<Record n=<Node element_id='4:f0adc280-ec80-4264-96b8-147afa2d1ff1:2' labels=frozenset({'WithVector'}) properties={'vector': [0.0, 1.0], 'id': 3}>>


In [15]:
records, summary, keys = driver.execute_query(
    '''
    call db.index.vector.queryNodes($index_name, 1, $vector) yield node
    return node
    ''',
    vector=[1.2, 0.3],
    index_name = index_name,
    routing_= RoutingControl.READ,
    database_= DB_NAME
)
for record in records:
    print(record)

<Record node=<Node element_id='4:f0adc280-ec80-4264-96b8-147afa2d1ff1:0' labels=frozenset({'WithVector'}) properties={'vector': [1.0, 0.0], 'id': 1}>>


In [23]:
records, summary, keys = driver.execute_query(
    '''
    cypher runtime=parallel
    unwind range(1,10000) as i
    with i
    call db.index.vector.queryNodes($index_name, 1, $vector) yield node
    return node
    ''',
    vector=[1.2, 0.3],
    index_name = index_name,
    routing_= RoutingControl.READ,
    database_= DB_NAME
)